In [108]:
import pandas as pd

In [109]:
df = pd.read_csv('./WEOApr2021all.csv')

/home/siddhant/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (52,53,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [110]:
df.columns

Index(['WEO Country Code', 'ISO', 'WEO Subject Code', 'Country',
       'Subject Descriptor', 'Subject Notes', 'Units', 'Scale',
       'Country/Series-specific Notes', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024', '2025', '2026', 'Estimates Start After',
       'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60',
       'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64'],
      dtype='object')

In [111]:
df[['Units','Scale']].dtypes

Units    object
Scale    object
dtype: object

In [112]:
len(' 2017 international dollar')

26

# Data Filtering and Preprocessing

## Choosing the required subjects

* Not all subjects are important for us.
* So to choose subjects first we are going to go through notes of each subject **to decide which subject to choose**.

In [113]:
subjects = dict()
for index, row in df.iterrows():
    if row['WEO Subject Code'] not in subjects:
        subjects[row['WEO Subject Code']] = { 
            'descriptior' : row['Subject Descriptor'], 
            'notes' : row['Subject Notes'],
            'unit' : row['Units']
        }

for key in subjects.keys():
    print()
    value = subjects[key]
    print(key)
    print('descriptior :', value['descriptior'])
    print('notes :', value['unit'])


NGDP_R
descriptior : Gross domestic product, constant prices
notes : National currency

NGDP_RPCH
descriptior : Gross domestic product, constant prices
notes : Percent change

NGDP
descriptior : Gross domestic product, current prices
notes : National currency

NGDPD
descriptior : Gross domestic product, current prices
notes : U.S. dollars

PPPGDP
descriptior : Gross domestic product, current prices
notes : Purchasing power parity

NGDP_D
descriptior : Gross domestic product, deflator
notes : Index

NGDPRPC
descriptior : Gross domestic product per capita, constant prices
notes : National currency

NGDPRPPPPC
descriptior : Gross domestic product per capita, constant prices
notes : Purchasing power parity

NGDPPC
descriptior : Gross domestic product per capita, current prices
notes : National currency

NGDPDPC
descriptior : Gross domestic product per capita, current prices
notes : U.S. dollars

PPPPC
descriptior : Gross domestic product per capita, current prices
notes : Purchasing power

In [114]:
# Choosen subjects

choosen_subjects = {
     # 'NGDP_R' : 'GDP (Constant)',
    'NGDP' : 'GDP (Current)',
     'NGDPD' : 'GDP (Current)', # USD
     # 'NGDPRPC' : 'GDP Per Capita (Constant)',
    'NGDPDPC' : 'GDP Per Capita (Current)', #USD
     # 'PPPSH' : 'GDP (PPP)',
    'NID_NGDP' : 'Total Investment',
    'NGSD_NGDP' : 'Gross national savings',
    'PCPI' : 'Inflation',
    # 'PCPIPCH' : 'Inflation (Avg Consumer Prices)',
    'TM_RPCH' : 'Imports',
    'TX_RPCH' : 'Exports',
    # 'LUR' : 'Unemployment rate',
    'LP' : 'Population',
    'GGR' : 'Goverment Revenue',
    'GGX' : 'Goverment Expenditure',
    'GGXCNL' : 'Goverment Borrowing',
    'GGXWDN' : 'Goverment Debt',
}

gdp_subjects = {
    'NGDP_R' : 'GDP (Constant)',
    'NGDPD' : 'GDP (Current)', # USD
    'NGDPRPC' : 'GDP Per Capita (Constant)',
    'NGDPDPC' : 'GDP Per Capita (Current)', #USD
    'PPPSH' : 'GDP (PPP)',
}

economic_subjects = {
    'NID_NGDP' : 'Total Investment',
    'NGSD_NGDP' : 'Gross national savings',
    'PCPI' : 'Inflation',
    'PCPIPCH' : 'Inflation (Avg Consumer Prices)',
    'TM_RPCH' : 'Imports',
    'TX_RPCH' : 'Exports',
    'LUR' : 'Unemployment rate',
    'LP' : 'Population',
}

gov_subjects = {
    'GGR' : 'Goverment Revenue',
    'GGX' : 'Goverment Expenditure',
    'GGXCNL' : 'Goverment Borrowing',
    'GGXWDN' : 'Goverment Debt',
}

In [115]:
subjects_df = df[df['WEO Subject Code'].isin(choosen_subjects.keys())][['WEO Subject Code','Subject Notes','Subject Descriptor','Units', 'Scale']]

In [116]:
df

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2026,Estimates Start After,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,512,AFG,NGDP_R,Afghanistan,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,1602.59,2019.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,512,AFG,NGDP_RPCH,Afghanistan,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",NaN,...,4.001,2019.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,512,AFG,NGDP,Afghanistan,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,2399.92,2019.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,512,AFG,NGDPD,Afghanistan,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",NaN,...,24.553,2019.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,512,AFG,PPPGDP,Afghanistan,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity,international dollars,Billions,"See notes for: Gross domestic product, curren...",...,107.246,113.629,2019.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8770,698,ZWE,GGXWDG,Zimbabwe,General government gross debt,Gross debt consists of all liabilities that re...,National currency,Billions,Source: Ministry of Finance or Treasury Latest...,Valuation of public debt: Current market valu...,...,2169.96,2357.230,2518.86,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
8771,698,ZWE,GGXWDG_NGDP,Zimbabwe,General government gross debt,Gross debt consists of all liabilities that re...,Percent of GDP,NaN,See notes for: General government gross debt ...,NaN,...,55.091,2018.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8772,698,ZWE,NGDP_FY,Zimbabwe,Gross domestic product corresponding to fiscal...,Gross domestic product corresponding to fiscal...,National currency,Billions,Source: Ministry of Finance or Treasury Latest...,Valuation of public debt: Current market valu...,...,3812.72,4217.200,4572.19,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
8773,698,ZWE,BCA,Zimbabwe,Current account balance,Current account is all transactions other than...,U.S. dollars,Billions,Source: Reserve Bank of Zimbabwe and Ministry ...,-0.301,...,0.203,2018.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Filtering the choosen subjects
* Also data of some initial years is not required

In [117]:
df_subject_code = df[df['WEO Subject Code'].isin(choosen_subjects.keys())]
df = df_subject_code.iloc[:, :56]
drop_columns = ['Subject Notes', 'Subject Descriptor', 'Country/Series-specific Notes'] + list(map(str, range(1980, 2002)))
df = df.drop(columns=drop_columns)
df

,WEO Country Code,ISO,WEO Subject Code,Country,Units,Scale,2002,2003,2004,2005,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
2,512,AFG,NGDP,Afghanistan,National currency,Billions,178.756,220.013,246.21,304.926,...,1285.46,1327.69,1469.6,1470.21,1599.35,1745.26,1896.29,2051.19,2218.82,2399.92
3,512,AFG,NGDPD,Afghanistan,U.S. dollars,Billions,4.367,4.553,5.146,6.167,...,18.91,18.401,18.876,19.132,19.938,21.201,22.01,22.754,23.896,24.553
9,512,AFG,NGDPDPC,Afghanistan,U.S. dollars,Units,233.433,233.755,254.259,294.396,...,636.693,582.323,586.204,580.817,591.667,614.991,624.1,630.714,647.452,650.3
14,512,AFG,NID_NGDP,Afghanistan,Percent of GDP,NaN,27.243,30.102,35.354,37.048,...,18.477,18.039,18.192,16.42,17.416,18.336,18.482,19.306,18.541,20.065
15,512,AFG,NGSD_NGDP,Afghanistan,Percent of GDP,NaN,61.151,59.718,72.57,67.274,...,26.063,30.197,29.897,27.132,27.391,26.613,26.222,26.565,23.941,23.739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8757,698,ZWE,LP,Zimbabwe,Persons,Millions,11.632,11.64,11.73,11.83,...,14.437,14.642,14.905,15.189,15.492,15.817,16.164,16.481,16.79,17.095
8758,698,ZWE,GGR,Zimbabwe,National currency,Billions,NaN,NaN,NaN,NaN,...,3.737,3.502,3.87,5.491,22.971,182.586,387.562,494.002,557.901,640.767
8760,698,ZWE,GGX,Zimbabwe,National currency,Billions,NaN,NaN,NaN,NaN,...,4.097,4.863,6.144,7.497,25.314,170.271,406.388,516.652,583.264,658.805
8762,698,ZWE,GGXCNL,Zimbabwe,National currency,Billions,NaN,NaN,NaN,NaN,...,-0.36,-1.361,-2.274,-2.006,-2.344,12.315,-18.825,-22.65,-25.363,-18.038


## Removing Null Values

In [118]:
# null elements
df.isna().sum()

WEO Country Code      0
ISO                   0
WEO Subject Code      0
Country               0
Units                 0
Scale               975
2002                325
2003                305
2004                270
2005                255
2006                248
2007                245
2008                238
2009                234
2010                230
2011                226
2012                227
2013                225
2014                212
2015                202
2016                201
2017                199
2018                201
2019                202
2020                206
2021                228
2022                228
2023                232
2024                234
2025                234
2026                234
dtype: int64

### Data Formatting

* Filling Nan data with '0'
* Converting data from string to float

In [119]:
# what fields are causing error for float to string conversion
for _ in df['2002']:
    try :
        st = float(_)
    except :
        print("Error for", _ , type(_))

Error for -- <class 'str'>
Error for -- <class 'str'>
Error for -- <class 'str'>
Error for -- <class 'str'>


In [120]:
# Checking how many fields are "--"
(df == "--").sum()

WEO Country Code    0
ISO                 0
WEO Subject Code    0
Country             0
Units               0
Scale               0
2002                4
2003                3
2004                1
2005                1
2006                2
2007                3
2008                1
2009                0
2010                2
2011                1
2012                0
2013                1
2014                0
2015                0
2016                0
2017                0
2018                0
2019                0
2020                1
2021                0
2022                0
2023                0
2024                0
2025                0
2026                0
dtype: int64

In [121]:
df.replace('--','0', inplace=True) 
df[list(map(str, range(2002, 2027)))] = df[list(map(str, range(2002, 2027)))].fillna('0')
df[list(map(str, range(2002, 2027)))] = df[list(map(str, range(2002, 2027)))].astype(float)
df.dtypes

WEO Country Code      int64
ISO                  object
WEO Subject Code     object
Country              object
Units                object
Scale                object
2002                float64
2003                float64
2004                float64
2005                float64
2006                float64
2007                float64
2008                float64
2009                float64
2010                float64
2011                float64
2012                float64
2013                float64
2014                float64
2015                float64
2016                float64
2017                float64
2018                float64
2019                float64
2020                float64
2021                float64
2022                float64
2023                float64
2024                float64
2025                float64
2026                float64
dtype: object

In [122]:
country_gdp = df[df['WEO Subject Code'] == 'NGDPD'][['ISO', 'Country', 'Units', 'Scale' ,'2021']]
country_gdp

,ISO,Country,Units,Scale,2021
3,AFG,Afghanistan,U.S. dollars,Billions,19.938
48,ALB,Albania,U.S. dollars,Billions,17.138
93,DZA,Algeria,U.S. dollars,Billions,151.459
138,AGO,Angola,U.S. dollars,Billions,66.493
183,ATG,Antigua and Barbuda,U.S. dollars,Billions,1.376
...,...,...,...,...,...
8553,VNM,Vietnam,U.S. dollars,Billions,354.868
8598,WBG,West Bank and Gaza,U.S. dollars,Billions,16.481
8643,YEM,Yemen,U.S. dollars,Billions,25.095
8688,ZMB,Zambia,U.S. dollars,Billions,18.955


#### Top 20 countries by GDP 

for year 2021

In [123]:
# country_gdp.isna().sum()
country_gdp.sort_values(by='2021', ascending=False)[:20]

,ISO,Country,Units,Scale,2021
8328,USA,United States,U.S. dollars,Billions,22675.270
1578,CHN,China,U.S. dollars,Billions,16642.320
3783,JPN,Japan,U.S. dollars,Billions,5378.140
2838,DEU,Germany,U.S. dollars,Billions,4319.290
8283,GBR,United Kingdom,U.S. dollars,Billions,3124.650
3423,IND,India,U.S. dollars,Billions,3049.700
2658,FRA,France,U.S. dollars,Billions,2938.270
3693,ITA,Italy,U.S. dollars,Billions,2106.290
1398,CAN,Canada,U.S. dollars,Billions,1883.490
4008,KOR,Korea,U.S. dollars,Billions,1806.710


In [124]:
# These are not g20 countries still roughly choosen
g20_countries = country_gdp.sort_values(by='2021', ascending=False)[:19]['ISO']
g20_countries

8328    USA
1578    CHN
3783    JPN
2838    DEU
8283    GBR
3423    IND
2658    FRA
3693    ITA
1398    CAN
4008    KOR
6393    RUS
363     AUS
1038    BRA
7158    ESP
4998    MEX
3468    IDN
5493    NLD
7518    CHE
6618    SAU
Name: ISO, dtype: object

In [125]:
country_gdp_constant = df[df['WEO Subject Code'] == 'NGDP'][['ISO', 'Country', 'Units', 'Scale' ,'2021']]
country_gdp_constant.sort_values(by='2021', ascending=False)[:20]

,ISO,Country,Units,Scale,2021
3512,IRN,Islamic Republic of Iran,National currency,Billions,47731850.96
3467,IDN,Indonesia,National currency,Billions,16427918.39
8552,VNM,Vietnam,National currency,Billions,8447685.12
4007,KOR,Korea,National currency,Billions,2002883.99
1622,COL,Colombia,National currency,Billions,1086308.71
8417,UZB,Uzbekistan,National currency,Billions,668939.45
3782,JPN,Japan,National currency,Billions,560059.19
3557,IRQ,Iraq,National currency,Billions,276562.29
6032,PRY,Paraguay,National currency,Billions,262388.41
3422,IND,India,National currency,Billions,228476.39


## Normalizing data

Most of the data is currently in the National Curreny wanted to convert it into USD.

In [126]:
x, y = df[df['WEO Subject Code'].isin(['NGDP','NGDPD'])].shape
x, y

(390, 31)

## Getting the currency exhange rate for each year

In [127]:
new_df_columns = ['WEO Country Code', 'ISO', 'Country',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026']
years = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026']



curr_ngdp = {}
curr_country = None

arr = []

# count = 0
for row_num, row in df[df['WEO Subject Code'].isin(['NGDP','NGDPD'])].iterrows():
    new_row = {}
    #print(row_num, row['WEO Subject Code'], curr_country, row['Country'])
    if row['WEO Subject Code'] == 'NGDP':
        for year in years:
            curr_ngdp[year] = row[year]
        curr_country = row['Country']
    elif row['WEO Subject Code'] == 'NGDPD' and curr_country == row['Country']:
        for col in ['WEO Country Code', 'ISO', 'Country']:
            new_row[col] = row[col]
        for year in years:
            if row[year] == 0:
                new_row[year] = 0
            else:
                new_row[year] = curr_ngdp[year]/row[year]
        
        arr.append(new_row)
#     count += 1
#     if(count > 3):
#         break

ExcDF = pd.DataFrame(arr)
ExcDF

,WEO Country Code,ISO,Country,2002,2003,2004,2005,2006,2007,2008,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,512,AFG,Afghanistan,40.933364,48.322644,47.844928,49.444787,49.937473,49.964353,50.258231,...,67.977790,72.153144,77.855478,76.845599,80.216170,82.319702,86.155838,90.146348,92.853197,97.744471
1,914,ALB,Albania,140.408004,120.787382,102.666110,99.871212,98.104204,90.430645,83.897213,...,118.796445,107.989701,109.873658,108.441936,102.549306,102.037118,101.406266,100.951021,100.905807,100.902816
2,612,DZA,Algeria,79.681471,77.394495,72.065114,73.276614,72.646825,69.292472,64.582663,...,110.901432,116.625705,119.414859,126.853507,137.438317,151.034069,162.274403,174.351226,187.327360,201.269489
3,614,AGO,Angola,43.529046,74.605064,83.541525,87.158854,80.368072,76.706708,75.033036,...,165.915627,252.856320,364.826624,578.247277,716.150422,788.641073,821.363212,838.655648,847.551173,867.376525
4,311,ATG,Antigua and Barbuda,2.698160,2.698598,2.698913,2.700587,2.700086,2.700992,2.701023,...,2.700272,2.700312,2.699759,2.699281,2.699855,2.700000,2.699177,2.699674,2.700565,2.699413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,582,VNM,Vietnam,15265.329758,15505.784843,15737.747030,15854.845554,15989.039395,16083.172744,16445.179925,...,22715.608646,23019.613573,23227.320422,23214.877780,23805.147604,24290.656684,24742.656543,25189.390756,25638.145593,26094.924356
191,487,WBG,West Bank and Gaza,4.737627,4.554183,4.482077,4.487515,4.456058,4.107806,3.588235,...,3.599578,3.590465,3.564433,3.442232,3.205691,3.194957,3.186979,3.181809,3.175747,3.169172
192,474,YEM,Yemen,175.629851,183.444558,184.849293,191.758307,197.242826,199.004203,199.763294,...,374.251945,493.005620,558.590039,740.999007,784.500100,927.454116,1007.098138,1095.967113,1166.561266,1234.043222
193,754,ZMB,Zambia,4.398426,4.733170,4.778974,4.465002,3.601630,4.001636,3.745060,...,9.517353,10.458156,12.889828,18.331318,22.902084,26.323953,28.849339,30.624421,32.104079,33.487141


In [128]:
ExcDF

,WEO Country Code,ISO,Country,2002,2003,2004,2005,2006,2007,2008,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,512,AFG,Afghanistan,40.933364,48.322644,47.844928,49.444787,49.937473,49.964353,50.258231,...,67.977790,72.153144,77.855478,76.845599,80.216170,82.319702,86.155838,90.146348,92.853197,97.744471
1,914,ALB,Albania,140.408004,120.787382,102.666110,99.871212,98.104204,90.430645,83.897213,...,118.796445,107.989701,109.873658,108.441936,102.549306,102.037118,101.406266,100.951021,100.905807,100.902816
2,612,DZA,Algeria,79.681471,77.394495,72.065114,73.276614,72.646825,69.292472,64.582663,...,110.901432,116.625705,119.414859,126.853507,137.438317,151.034069,162.274403,174.351226,187.327360,201.269489
3,614,AGO,Angola,43.529046,74.605064,83.541525,87.158854,80.368072,76.706708,75.033036,...,165.915627,252.856320,364.826624,578.247277,716.150422,788.641073,821.363212,838.655648,847.551173,867.376525
4,311,ATG,Antigua and Barbuda,2.698160,2.698598,2.698913,2.700587,2.700086,2.700992,2.701023,...,2.700272,2.700312,2.699759,2.699281,2.699855,2.700000,2.699177,2.699674,2.700565,2.699413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,582,VNM,Vietnam,15265.329758,15505.784843,15737.747030,15854.845554,15989.039395,16083.172744,16445.179925,...,22715.608646,23019.613573,23227.320422,23214.877780,23805.147604,24290.656684,24742.656543,25189.390756,25638.145593,26094.924356
191,487,WBG,West Bank and Gaza,4.737627,4.554183,4.482077,4.487515,4.456058,4.107806,3.588235,...,3.599578,3.590465,3.564433,3.442232,3.205691,3.194957,3.186979,3.181809,3.175747,3.169172
192,474,YEM,Yemen,175.629851,183.444558,184.849293,191.758307,197.242826,199.004203,199.763294,...,374.251945,493.005620,558.590039,740.999007,784.500100,927.454116,1007.098138,1095.967113,1166.561266,1234.043222
193,754,ZMB,Zambia,4.398426,4.733170,4.778974,4.465002,3.601630,4.001636,3.745060,...,9.517353,10.458156,12.889828,18.331318,22.902084,26.323953,28.849339,30.624421,32.104079,33.487141


In [130]:
df_usd = df.copy()

for i, row in df[df['Units'] == 'National currency'].iterrows():
    for year in list(map(str, range(2002, 2027))):
        exc = ExcDF[ExcDF['Country'] == row['Country']][year].iloc[0]
        val = row[year]
        #         print(type(exc), exc)
        # print(exc.iloc[0])
        if exc != 0:
            # print(i, year, val/exc)
            df_usd.at[i, year] = val/exc
            df_usd.at[i, 'Units'] = 'U.S. dollars'
        else:
            df_usd.at[i, year] = -1
    
df_usd

<class 'numpy.float64'> 40.93336386535379
<class 'numpy.float64'> 48.32264441027894
<class 'numpy.float64'> 47.84492809949476
<class 'numpy.float64'> 49.444786768282796
<class 'numpy.float64'> 49.93747292418773
<class 'numpy.float64'> 49.964352501168776
<class 'numpy.float64'> 50.25823055258813
<class 'numpy.float64'> 50.32545997016409
<class 'numpy.float64'> 46.44430668841762
<class 'numpy.float64'> 46.74242593627725
<class 'numpy.float64'> 50.93332676292317
<class 'numpy.float64'> 55.37084779375309
<class 'numpy.float64'> 57.33171795493094
<class 'numpy.float64'> 60.661226508407516
<class 'numpy.float64'> 67.9626542180727
<class 'numpy.float64'> 67.97778952934955
<class 'numpy.float64'> 72.15314385087767
<class 'numpy.float64'> 77.85547785547784
<class 'numpy.float64'> 76.84559899644574
<class 'numpy.float64'> 80.21617012739492
<class 'numpy.float64'> 82.31970190085373
<class 'numpy.float64'> 86.15583825533848
<class 'numpy.float64'> 90.14634789487562
<class 'numpy.float64'> 92.85319

,WEO Country Code,ISO,WEO Subject Code,Country,Units,Scale,2002,2003,2004,2005,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
2,512,AFG,NGDP,Afghanistan,U.S. dollars,Billions,4.367,4.553,5.146,6.167,...,18.910000,18.401000,18.876000,19.132000,19.938000,21.201000,22.010000,22.754000,23.896000,24.553000
3,512,AFG,NGDPD,Afghanistan,U.S. dollars,Billions,4.367,4.553,5.146,6.167,...,18.910000,18.401000,18.876000,19.132000,19.938000,21.201000,22.010000,22.754000,23.896000,24.553000
9,512,AFG,NGDPDPC,Afghanistan,U.S. dollars,Units,233.433,233.755,254.259,294.396,...,636.693000,582.323000,586.204000,580.817000,591.667000,614.991000,624.100000,630.714000,647.452000,650.300000
14,512,AFG,NID_NGDP,Afghanistan,Percent of GDP,NaN,27.243,30.102,35.354,37.048,...,18.477000,18.039000,18.192000,16.420000,17.416000,18.336000,18.482000,19.306000,18.541000,20.065000
15,512,AFG,NGSD_NGDP,Afghanistan,Percent of GDP,NaN,61.151,59.718,72.570,67.274,...,26.063000,30.197000,29.897000,27.132000,27.391000,26.613000,26.222000,26.565000,23.941000,23.739000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8757,698,ZWE,LP,Zimbabwe,Persons,Millions,11.632,11.640,11.730,11.830,...,14.437000,14.642000,14.905000,15.189000,15.492000,15.817000,16.164000,16.481000,16.790000,17.095000
8758,698,ZWE,GGR,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,2.981374,1.718253,0.459814,0.105966,0.259216,1.823934,3.422930,3.884284,4.085171,4.490508
8760,698,ZWE,GGX,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,3.268581,2.386026,0.730000,0.144678,0.285656,1.700914,3.589201,4.062378,4.270889,4.616919
8762,698,ZWE,GGXCNL,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,-0.287208,-0.667773,-0.270186,-0.038712,-0.026451,0.123020,-0.166262,-0.178094,-0.185718,-0.126411


In [135]:
df_usd['Units'].value_counts()

U.S. dollars      1365
Percent of GDP     390
Percent change     390
Index              195
Persons            195
Name: Units, dtype: int64

In [136]:
df_usd['Scale'].value_counts()

Billions    1170
Units        195
Millions     195
Name: Scale, dtype: int64

In [137]:
country_gdp_constant = df[df['WEO Subject Code'] == 'NGDP'][['ISO', 'Country', 'Units', 'Scale' ,'2023']]
country_gdp_constant.sort_values(by='2023', ascending=False)[:20]

,ISO,Country,Units,Scale,2023
3512,IRN,Islamic Republic of Iran,National currency,Billions,77784173.97
3467,IDN,Indonesia,National currency,Billions,19517735.04
8552,VNM,Vietnam,National currency,Billions,10592925.09
4007,KOR,Korea,National currency,Billions,2153283.81
1622,COL,Colombia,National currency,Billions,1246151.62
8417,UZB,Uzbekistan,National currency,Billions,894059.06
3782,JPN,Japan,National currency,Billions,586464.08
3557,IRQ,Iraq,National currency,Billions,314582.56
6032,PRY,Paraguay,National currency,Billions,301150.22
3422,IND,India,National currency,Billions,283902.41


In [138]:
country_gdp_constant = df[df['WEO Subject Code'] == 'NGDPD'][['ISO', 'Country', 'Units', 'Scale' ,'2023']]
country_gdp_constant.sort_values(by='2023', ascending=False)[:20]

,ISO,Country,Units,Scale,2023
8328,USA,United States,U.S. dollars,Billions,24892.610
1578,CHN,China,U.S. dollars,Billions,19423.480
3783,JPN,Japan,U.S. dollars,Billions,5977.360
2838,DEU,Germany,U.S. dollars,Billions,4757.650
3423,IND,India,U.S. dollars,Billions,3591.030
8283,GBR,United Kingdom,U.S. dollars,Billions,3490.270
2658,FRA,France,U.S. dollars,Billions,3242.500
3693,ITA,Italy,U.S. dollars,Billions,2295.730
1398,CAN,Canada,U.S. dollars,Billions,2142.960
4008,KOR,Korea,U.S. dollars,Billions,1967.850


# Saving the data

### DF_USD

In [139]:
df_usd

,WEO Country Code,ISO,WEO Subject Code,Country,Units,Scale,2002,2003,2004,2005,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
2,512,AFG,NGDP,Afghanistan,U.S. dollars,Billions,4.367,4.553,5.146,6.167,...,18.910000,18.401000,18.876000,19.132000,19.938000,21.201000,22.010000,22.754000,23.896000,24.553000
3,512,AFG,NGDPD,Afghanistan,U.S. dollars,Billions,4.367,4.553,5.146,6.167,...,18.910000,18.401000,18.876000,19.132000,19.938000,21.201000,22.010000,22.754000,23.896000,24.553000
9,512,AFG,NGDPDPC,Afghanistan,U.S. dollars,Units,233.433,233.755,254.259,294.396,...,636.693000,582.323000,586.204000,580.817000,591.667000,614.991000,624.100000,630.714000,647.452000,650.300000
14,512,AFG,NID_NGDP,Afghanistan,Percent of GDP,NaN,27.243,30.102,35.354,37.048,...,18.477000,18.039000,18.192000,16.420000,17.416000,18.336000,18.482000,19.306000,18.541000,20.065000
15,512,AFG,NGSD_NGDP,Afghanistan,Percent of GDP,NaN,61.151,59.718,72.570,67.274,...,26.063000,30.197000,29.897000,27.132000,27.391000,26.613000,26.222000,26.565000,23.941000,23.739000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8757,698,ZWE,LP,Zimbabwe,Persons,Millions,11.632,11.640,11.730,11.830,...,14.437000,14.642000,14.905000,15.189000,15.492000,15.817000,16.164000,16.481000,16.790000,17.095000
8758,698,ZWE,GGR,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,2.981374,1.718253,0.459814,0.105966,0.259216,1.823934,3.422930,3.884284,4.085171,4.490508
8760,698,ZWE,GGX,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,3.268581,2.386026,0.730000,0.144678,0.285656,1.700914,3.589201,4.062378,4.270889,4.616919
8762,698,ZWE,GGXCNL,Zimbabwe,U.S. dollars,Billions,0.000,0.000,0.000,0.000,...,-0.287208,-0.667773,-0.270186,-0.038712,-0.026451,0.123020,-0.166262,-0.178094,-0.185718,-0.126411


In [141]:
df_usd_g20 = df_usd[df_usd['ISO'].isin(g20_countries)]
df_usd_g20

,WEO Country Code,ISO,WEO Subject Code,Country,Units,Scale,2002,2003,2004,2005,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
362,193,AUS,NGDP,Australia,U.S. dollars,Billions,424.425,539.562,656.735,734.057,...,1385.190,1421.300,1391.540,1359.330,1617.540,1693.020,1772.980,1848.920,1929.060,2017.320
363,193,AUS,NGDPD,Australia,U.S. dollars,Billions,424.425,539.562,656.735,734.057,...,1385.190,1421.300,1391.540,1359.330,1617.540,1693.020,1772.980,1848.920,1929.060,2017.320
369,193,AUS,NGDPDPC,Australia,U.S. dollars,Units,21648.340,27213.270,32761.410,36139.870,...,55914.690,56465.240,54464.060,52824.820,62723.500,65183.150,67474.300,69484.490,71589.230,73928.250
374,193,AUS,NID_NGDP,Australia,Percent of GDP,NaN,25.289,26.835,27.304,27.930,...,24.271,24.134,22.544,22.317,22.288,22.465,22.650,22.910,23.102,23.212
375,193,AUS,NGSD_NGDP,Australia,Percent of GDP,NaN,21.515,21.472,21.071,21.813,...,21.972,21.900,23.307,24.571,24.651,23.431,22.970,22.582,22.328,22.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8352,111,USA,LP,United States,Persons,Millions,288.019,290.733,293.389,296.115,...,325.143,326.882,328.461,330.086,331.952,333.868,335.795,337.734,339.684,341.645
8353,111,USA,GGR,United States,U.S. dollars,Billions,3266.460,3353.770,3601.490,4023.390,...,6006.550,6186.230,6428.920,6350.410,6804.760,7488.790,7792.100,8024.390,8315.630,8753.130
8355,111,USA,GGX,United States,U.S. dollars,Billions,3683.340,3899.590,4119.150,4423.470,...,6908.790,7301.430,7647.500,9666.480,10213.310,8960.290,8942.180,9233.090,9657.290,10060.550
8357,111,USA,GGXCNL,United States,U.S. dollars,Billions,-416.881,-545.819,-517.661,-400.079,...,-902.232,-1115.200,-1218.580,-3316.070,-3408.540,-1471.500,-1150.080,-1208.700,-1341.660,-1307.420


In [142]:
df_usd_g20[df_usd_g20['WEO Subject Code'] == "NGDP"].sort_values(by=['2023'])

,WEO Country Code,ISO,WEO Subject Code,Country,Units,Scale,2002,2003,2004,2005,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
6617,456,SAU,NGDP,Saudi Arabia,U.S. dollars,Billions,189.606,215.808,258.742,328.206,...,688.586,786.522,792.967,701.467,804.921,827.133,854.717,889.176,923.844,966.264
7517,146,CHE,NGDP,Switzerland,U.S. dollars,Billions,310.611,363.391,405.059,420.153,...,704.772,736.433,732.187,747.427,824.734,874.157,905.181,949.873,988.213,1040.200
5492,138,NLD,NGDP,Netherlands,U.S. dollars,Billions,473.527,579.925,658.081,685.727,...,833.575,914.458,907.151,909.503,1012.600,1073.230,1113.020,1150.970,1189.540,1229.130
4997,273,MEX,NGDP,Mexico,U.S. dollars,Billions,772.110,729.335,782.243,877.477,...,1158.910,1222.350,1268.870,1076.160,1192.480,1251.090,1307.810,1368.870,1430.840,1494.680
3467,536,IDN,NGDP,Indonesia,U.S. dollars,Billions,212.807,255.428,279.556,310.815,...,1015.490,1042.710,1120.040,1059.640,1158.780,1256.280,1360.260,1458.740,1561.950,1672.540
7157,184,ESP,NGDP,Spain,U.S. dollars,Billions,708.256,907.264,1068.570,1154.350,...,1312.080,1422.800,1393.640,1278.210,1461.550,1571.100,1644.730,1715.980,1769.030,1824.050
1037,223,BRA,NGDP,Brazil,U.S. dollars,Billions,509.798,558.232,669.290,891.633,...,2063.520,1916.930,1877.110,1434.080,1491.770,1636.410,1769.710,1922.410,2050.390,2172.080
362,193,AUS,NGDP,Australia,U.S. dollars,Billions,424.425,539.562,656.735,734.057,...,1385.190,1421.300,1391.540,1359.330,1617.540,1693.020,1772.980,1848.920,1929.060,2017.320
6392,922,RUS,NGDP,Russia,U.S. dollars,Billions,370.062,461.518,633.294,817.717,...,1575.140,1653.010,1689.300,1473.580,1710.730,1777.100,1826.960,1879.530,1939.000,2004.150
4007,542,KOR,NGDP,Korea,U.S. dollars,Billions,626.989,702.696,792.532,934.708,...,1623.070,1725.370,1646.740,1630.870,1806.710,1885.680,1967.850,2052.930,2137.990,2220.720


In [143]:
df_usd_g20['ISO'].value_counts()

AUS    13
KOR    13
GBR    13
CHE    13
ESP    13
SAU    13
RUS    13
NLD    13
MEX    13
JPN    13
BRA    13
ITA    13
IDN    13
IND    13
DEU    13
FRA    13
CHN    13
CAN    13
USA    13
Name: ISO, dtype: int64

In [28]:
df_usd.to_csv("world-economy-data-usd.csv", index=True, header=True, encoding="utf-8")

### Subjects Data Frame

In [29]:
# subjects_df = df[df['WEO Subject Code'].isin(choosen_subjects.keys())][['WEO Subject Code','Subject Notes','Subject Descriptor','Units', 'Scale']]
subjects_df.to_csv("subjects.csv", index=True, header=True, encoding="utf-8")

# Adding data into postgresql database

In [153]:
from sys import exit
from os import environ
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData,  Table, Column, Integer, String, Float, inspect, Index, insert, Text

In [154]:
df_usd_g20['Units'].value_counts()

U.S. dollars      133
Percent of GDP     38
Percent change     38
Index              19
Persons            19
Name: Units, dtype: int64

In [155]:
load_dotenv("../.env")

postgresql_url = environ.get("POSTGRES_CONNECT_URL")
if postgresql_url is None:
    print("No variable named POSTGRES_CONNECT_URL found in ../.env file")
    exit()

# print(postgresql_url)
engine = create_engine(postgresql_url)

meta = MetaData(engine)

world_data_table = Table('world_data', meta,                                     # 'public', 
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("iso", String(4), nullable=False),
    Column("country", String(40), nullable=False),
    Column("units", String(30)),
    Column("country_code", Integer, nullable=False),
    Column("subject_code", String(20), nullable=False),
    Column("scale", String(30)),
              
    Column("year_2002", Float),
    Column("year_2003", Float),
    Column("year_2004", Float),
    Column("year_2005", Float),
    Column("year_2006", Float),
    Column("year_2007", Float),
    Column("year_2008", Float),
    Column("year_2009", Float),
    Column("year_2010", Float),
    Column("year_2011", Float),
    Column("year_2012", Float),
    Column("year_2013", Float),
    Column("year_2014", Float),
    Column("year_2015", Float),
    Column("year_2016", Float),
    Column("year_2017", Float),
    Column("year_2018", Float),
    Column("year_2019", Float),
    Column("year_2020", Float),
    Column("year_2021", Float),
    Column("year_2022", Float),
    Column("year_2023", Float),
    Column("year_2024", Float),
    Column("year_2025", Float),
    Column("year_2026", Float),
)

pandas_column_to_sql_column = {
    'WEO Country Code' : 'country_code',
    'ISO' : 'iso',
    'WEO Subject Code' : 'subject_code',
    'Country' : 'country',
    'Units' : 'units',
    'Scale' : 'scale',
    
    '2002' : 'year_2002',
    '2003' : 'year_2003',
    '2004' : 'year_2004',
    '2005' : 'year_2005',
    '2006' : 'year_2006',
    '2007' : 'year_2007',
    '2008' : 'year_2008',
    '2009' : 'year_2009',
    '2010' : 'year_2010',
    '2011' : 'year_2011',
    '2012' : 'year_2012',
    '2013' : 'year_2013',
    '2014' : 'year_2014',
    '2015' : 'year_2015',
    '2016' : 'year_2016',
    '2017' : 'year_2017',
    '2018' : 'year_2018',
    '2019' : 'year_2019',
    '2020' : 'year_2020',
    '2021' : 'year_2021',
    '2022' : 'year_2022',
    '2023' : 'year_2023',
    '2024' : 'year_2024',
    '2025' : 'year_2025',
    '2026' : 'year_2026',
}


iso_index = Index(
    "iso_index", world_data_table.c.iso)
country_code_index = Index(
    "country_code_index", world_data_table.c.country_code)
subject_code_index = Index(
    "subject_code_index", world_data_table.c.subject_code)
country_index = Index(
    "country_index", world_data_table.c.country)

if not inspect(engine).has_table('world_data'):
    meta.create_all()
    
    # Very huge data of over 8700 rows try g20 countires data insted
    # for i, row in df_usd.iterrows():
    #     # new_entry = {}
    #     stmt_columns = ""
    #     stmt_values = ""
    #     for col in df_usd.columns:
    #         if pandas_column_to_sql_column[col] is not None:
    #             # new_entry[pandas_column_to_sql_column[col]] = row[col]
    #             stmt_columns += pandas_column_to_sql_column[col] + ", "
    #             if pd.isna(row[col]):
    #                 stmt_values += "NULL, "
    #             elif type(row[col]) is str:
    #                 stmt_values +=  "'" + str(row[col]).replace("''","''").replace("'", "''") + "', "
    #             else:
    #                 stmt_values += str(row[col]) + ", "
    #     if len(stmt_columns) > 2:
    #         # print(stmt_values[:-2])
    #         command = f"INSERT INTO world_data ({stmt_columns[:-2]}) VALUES ({stmt_values[:-2]});"
    #         engine.execute(command)
    
    for i, row in df_usd_g20.iterrows():
        # new_entry = {}
        stmt_columns = ""
        stmt_values = ""
        for col in df_usd_g20.columns:
            if pandas_column_to_sql_column[col] is not None:
                # new_entry[pandas_column_to_sql_column[col]] = row[col]
                stmt_columns += pandas_column_to_sql_column[col] + ", "
                if pd.isna(row[col]):
                    stmt_values += "NULL, "
                elif type(row[col]) is str:
                    stmt_values +=  "'" + str(row[col]).replace("''","''").replace("'", "''") + "', "
                else:
                    stmt_values += str(row[col]) + ", "
        if len(stmt_columns) > 2:
            # print(stmt_values[:-2])
            command = f"INSERT INTO world_data ({stmt_columns[:-2]}) VALUES ({stmt_values[:-2]});"
            engine.execute(command)

    print("DOne!")
else:
    print("table already exist")

DOne!


In [156]:
len('Percent of total labor force')

28

In [157]:
df_usd['Units'].value_counts()

U.S. dollars      1365
Percent of GDP     390
Percent change     390
Index              195
Persons            195
Name: Units, dtype: int64

In [158]:
len('Democratic Republic of the Congo')

32

In [159]:
len('Bosnia and Herzegovina')

22

In [160]:
df_usd['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Democratic Republic of the Congo', 'Republic of Congo',
       'Costa Rica', "C�te d'Ivoire", 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'The Gambia', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Gui

### Subjects Data

In [161]:
subjects_df = subjects_df.groupby('WEO Subject Code').first().reset_index()
subjects_df

,WEO Subject Code,Subject Notes,Subject Descriptor,Units,Scale
0,GGR,"Revenue consists of taxes, social contribution...",General government revenue,National currency,Billions
1,GGX,Total expenditure consists of total expense an...,General government total expenditure,National currency,Billions
2,GGXCNL,Net lending (+)/ borrowing (-) is calculated a...,General government net lending/borrowing,National currency,Billions
3,GGXWDN,Net debt is calculated as gross debt minus fin...,General government net debt,National currency,Billions
4,LP,"For census purposes, the total population of t...",Population,Persons,Millions
5,NGDP,Expressed in billions of national currency uni...,"Gross domestic product, current prices",National currency,Billions
6,NGDPD,Values are based upon GDP in national currency...,"Gross domestic product, current prices",U.S. dollars,Billions
7,NGDPDPC,GDP is expressed in current U.S. dollars per p...,"Gross domestic product per capita, current prices",U.S. dollars,Units
8,NGSD_NGDP,Expressed as a ratio of gross national savings...,Gross national savings,Percent of GDP,None
9,NID_NGDP,Expressed as a ratio of total investment in cu...,Total investment,Percent of GDP,None


In [162]:
subjects_df.columns

Index(['WEO Subject Code', 'Subject Notes', 'Subject Descriptor', 'Units',
       'Scale'],
      dtype='object')

In [163]:
for _ in subjects_df[['Scale']]:
    print(_)

Scale


In [164]:
def get_string_max_length(seq):
    max_len = 0
    ans = ""
    for _ in seq:
        if _ is not None and len(_) > max_len:
            max_len = len(_)
            ans = _
    return ans, max_len

for col in subjects_df.columns:
    print(col, ":\n", get_string_max_length(subjects_df[col]))
    print()
    print()

WEO Subject Code :
 ('NGSD_NGDP', 9)


Subject Notes :
 ("Expressed in averages for the year, not end-of-period data. A consumer price index (CPI) measures changes in the prices of goods and services that households consume. Such changes affect the real purchasing power of consumers' incomes and their welfare. As the prices of different goods and services do not all change at the same rate, a price index can only reflect their average movement. A price index is typically assigned a value of unity, or 100, in some reference period and the values of the index for other periods of time are intended to indicate the average proportionate, or percentage, change in prices from this price reference period. Price indices can also be used to measure differences in price levels between different cities, regions or countries at the same point in time. [CPI Manual 2004, Introduction] For euro countries, consumer prices are calculated based on harmonized prices. For more information see http://epp.e

In [165]:
load_dotenv("../.env")

postgresql_url = environ.get("POSTGRES_CONNECT_URL")
if postgresql_url is None:
    print("No variable named POSTGRES_CONNECT_URL found in ../.env file")
    exit()

# print(postgresql_url)
engine = create_engine(postgresql_url)

meta = MetaData(engine)

subject_data_table = Table('subject_data', meta,                                     # 'public', 
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("subject_code", String(10), nullable=False),
    Column("description", Text),
    Column("notes", Text),
    Column("units", String(30)),
    Column("scale", String(8)),
)

pandas_column_to_sql_column = {
    'WEO Subject Code' : 'subject_code',
    'Units' : 'units',
    'Scale' : 'scale',
    'Subject Notes' : 'notes',
    'Subject Descriptor' : 'description'
}


if not inspect(engine).has_table('subject_data'):
    meta.create_all()
    
    for i, row in subjects_df.iterrows():
        # new_entry = {}
        stmt_columns = ""
        stmt_values = ""
        for col in subjects_df.columns:
            if pandas_column_to_sql_column[col] is not None:
                # new_entry[pandas_column_to_sql_column[col]] = row[col]
                stmt_columns += pandas_column_to_sql_column[col] + ", "
                if pd.isna(row[col]):
                    stmt_values += "NULL, "
                elif type(row[col]) is str:
                    stmt_values +=  "'" + str(row[col]).replace("''","''").replace("'", "''") + "', "
                else:
                    stmt_values += str(row[col]) + ", "
        if len(stmt_columns) > 2:
            # print(stmt_values[:-2])
            command = f"INSERT INTO subject_data ({stmt_columns[:-2]}) VALUES ({stmt_values[:-2]});"
            engine.execute(command)
    # subjects_df.to_sql('world_data', con=engine)
    print("DOne!")
else:
    print("table already exist")

DOne!
